In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(" Kütüphaneler yüklendi!")
print("\n" + "=" * 80)
print(" VERİ BİRLEŞTİRME SÜRECİ ")
print("=" * 80)

 Kütüphaneler yüklendi!

 VERİ BİRLEŞTİRME SÜRECİ 


In [3]:
PROCESSED_PATH = '../data/processed/'

print("\n Temizlenmiş veri setleri yükleniyor...")

orders = pd.read_csv(f'{PROCESSED_PATH}orders_clean.csv')
customers = pd.read_csv(f'{PROCESSED_PATH}customers_clean.csv')
order_items = pd.read_csv(f'{PROCESSED_PATH}order_items_clean.csv')
products = pd.read_csv(f'{PROCESSED_PATH}products_clean.csv')
sellers = pd.read_csv(f'{PROCESSED_PATH}sellers_clean.csv')
payments = pd.read_csv(f'{PROCESSED_PATH}payments_clean.csv')
reviews = pd.read_csv(f'{PROCESSED_PATH}reviews_clean.csv')
geolocation = pd.read_csv(f'{PROCESSED_PATH}geolocation_clean.csv')

# Tarih kolonlarını tekrar datetime'a çevir
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date',
             'order_estimated_delivery_date']
for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])

print(" Tüm veri setleri yüklendi!")
print(f"\n Yüklenen veri setleri:")
print(f"   - Orders: {orders.shape}")
print(f"   - Customers: {customers.shape}")
print(f"   - Order Items: {order_items.shape}")
print(f"   - Products: {products.shape}")
print(f"   - Sellers: {sellers.shape}")
print(f"   - Payments: {payments.shape}")
print(f"   - Reviews: {reviews.shape}")
print(f"   - Geolocation: {geolocation.shape}")


 Temizlenmiş veri setleri yükleniyor...
 Tüm veri setleri yüklendi!

 Yüklenen veri setleri:
   - Orders: (96470, 14)
   - Customers: (99441, 5)
   - Order Items: (112650, 8)
   - Products: (32951, 11)
   - Sellers: (3095, 4)
   - Payments: (103877, 5)
   - Reviews: (99224, 8)
   - Geolocation: (19015, 5)


In [4]:
print("\n" + "=" * 80)
print("ADIM 1: ORDERS + ORDER_ITEMS Birleştirme")
print("=" * 80)

print("\nÖnce:")
print(f"  Orders satır: {len(orders):,}")
print(f"  Order Items satır: {len(order_items):,}")

# Orders ile Order Items'ı birleştir (LEFT JOIN)
df = pd.merge(
    orders,
    order_items,
    on='order_id',
    how='left'
)

print(f"\nSonra:")
print(f"  Birleştirilmiş satır: {len(df):,}")
print(f"  Kolonlar: {df.shape[1]}")

# Kontrol: Her satırda order_id olmalı
print(f"\n✓ Kontrol: Eksik order_id var mı? {df['order_id'].isnull().sum()}")


ADIM 1: ORDERS + ORDER_ITEMS Birleştirme

Önce:
  Orders satır: 96,470
  Order Items satır: 112,650

Sonra:
  Birleştirilmiş satır: 110,189
  Kolonlar: 21

✓ Kontrol: Eksik order_id var mı? 0


In [5]:
print("\n" + "=" * 80)
print("ADIM 2: PRODUCTS Bilgisini Ekle")
print("=" * 80)

print(f"\nÖnce: {len(df):,} satır")

# Products bilgisini ekle (LEFT JOIN)
df = pd.merge(
    df,
    products,
    on='product_id',
    how='left'
)

print(f"Sonra: {len(df):,} satır")
print(f"Kolonlar: {df.shape[1]}")

# Kaç ürün kategorisi var?
print(f"\n✓ Benzersiz kategori: {df['product_category_name'].nunique()}")


ADIM 2: PRODUCTS Bilgisini Ekle

Önce: 110,189 satır
Sonra: 110,189 satır
Kolonlar: 31

✓ Benzersiz kategori: 74


In [6]:
print("\n" + "=" * 80)
print("ADIM 3: SELLERS Bilgisini Ekle")
print("=" * 80)

print(f"\nÖnce: {len(df):,} satır")

# Sellers bilgisini ekle (LEFT JOIN)
# Kolon isimlerini değiştirerek ekle (karışmasın diye)
df = pd.merge(
    df,
    sellers.add_prefix('seller_'),
    left_on='seller_id',
    right_on='seller_seller_id',
    how='left'
)

# Gereksiz kolonu sil
df.drop('seller_seller_id', axis=1, inplace=True)

print(f"Sonra: {len(df):,} satır")
print(f"Kolonlar: {df.shape[1]}")
print(f"\n✓ Benzersiz satıcı: {df['seller_id'].nunique()}")


ADIM 3: SELLERS Bilgisini Ekle

Önce: 110,189 satır
Sonra: 110,189 satır
Kolonlar: 34

✓ Benzersiz satıcı: 2970


In [7]:
print("\n" + "=" * 80)
print("ADIM 4: CUSTOMERS Bilgisini Ekle")
print("=" * 80)

print(f"\nÖnce: {len(df):,} satır")

# Customers bilgisini ekle (LEFT JOIN)
df = pd.merge(
    df,
    customers,
    on='customer_id',
    how='left'
)

print(f"Sonra: {len(df):,} satır")
print(f"Kolonlar: {df.shape[1]}")
print(f"\n✓ Benzersiz müşteri: {df['customer_unique_id'].nunique()}")


ADIM 4: CUSTOMERS Bilgisini Ekle

Önce: 110,189 satır
Sonra: 110,189 satır
Kolonlar: 38

✓ Benzersiz müşteri: 93350


In [8]:
print("\n" + "=" * 80)
print("ADIM 5: PAYMENTS Bilgisini Ekle")
print("=" * 80)

# Bir siparişte birden fazla ödeme olabilir, topla
payments_agg = payments.groupby('order_id').agg({
    'payment_sequential': 'max',  # En yüksek payment numarası
    'payment_type': lambda x: ', '.join(x.unique()),  # Tüm ödeme tipleri
    'payment_installments': 'max',  # En yüksek taksit
    'payment_value': 'sum'  # Toplam ödeme
}).reset_index()

# Kolon isimlerini değiştir
payments_agg.rename(columns={
    'payment_sequential': 'total_payments',
    'payment_type': 'payment_types',
    'payment_installments': 'max_installments',
    'payment_value': 'total_payment_value'
}, inplace=True)

print(f"\nPayments toplamı:")
print(f"  Benzersiz sipariş: {len(payments_agg):,}")
print(f"  Toplam ödeme: ${payments_agg['total_payment_value'].sum():,.2f}")

# Payments bilgisini ekle (LEFT JOIN)
df = pd.merge(
    df,
    payments_agg,
    on='order_id',
    how='left'
)

print(f"\nBirleştirme sonrası:")
print(f"  Satır: {len(df):,}")
print(f"  Kolonlar: {df.shape[1]}")


ADIM 5: PAYMENTS Bilgisini Ekle

Payments toplamı:
  Benzersiz sipariş: 99,437
  Toplam ödeme: $16,008,872.12

Birleştirme sonrası:
  Satır: 110,189
  Kolonlar: 42


In [9]:
print("\n" + "=" * 80)
print("ADIM 6: REVIEWS Bilgisini Ekle")
print("=" * 80)

# Bir siparişte birden fazla review olabilir (genelde 1), ortalama al
reviews_agg = reviews.groupby('order_id').agg({
    'review_score': 'mean',
    'review_comment_message': 'first',  # İlk yorumu al
    'comment_length': 'mean'
}).reset_index()

reviews_agg.rename(columns={
    'review_score': 'avg_review_score',
    'review_comment_message': 'review_message',
    'comment_length': 'avg_comment_length'
}, inplace=True)

print(f"\nReviews toplamı:")
print(f"  Benzersiz sipariş: {len(reviews_agg):,}")
print(f"  Ortalama puan: {reviews_agg['avg_review_score'].mean():.2f}/5.00")

# Reviews bilgisini ekle (LEFT JOIN)
df = pd.merge(
    df,
    reviews_agg,
    on='order_id',
    how='left'
)

print(f"\nBirleştirme sonrası:")
print(f"  Satır: {len(df):,}")
print(f"  Kolonlar: {df.shape[1]}")


ADIM 6: REVIEWS Bilgisini Ekle

Reviews toplamı:
  Benzersiz sipariş: 98,673
  Ortalama puan: 4.09/5.00

Birleştirme sonrası:
  Satır: 110,189
  Kolonlar: 45


In [10]:
print("\n" + "=" * 80)
print("ADIM 7: GEOLOCATION Bilgisini Ekle")
print("=" * 80)

# Customer lokasyonu için geolocation ekle
df = pd.merge(
    df,
    geolocation.add_prefix('customer_geo_'),
    left_on='customer_zip_code_prefix',
    right_on='customer_geo_geolocation_zip_code_prefix',
    how='left'
)

# Gereksiz kolonu sil
df.drop('customer_geo_geolocation_zip_code_prefix', axis=1, inplace=True)

# Seller lokasyonu için geolocation ekle
df = pd.merge(
    df,
    geolocation.add_prefix('seller_geo_'),
    left_on='seller_seller_zip_code_prefix',
    right_on='seller_geo_geolocation_zip_code_prefix',
    how='left'
)

# Gereksiz kolonu sil
df.drop('seller_geo_geolocation_zip_code_prefix', axis=1, inplace=True)

print(f"\nBirleştirme sonrası:")
print(f"  Satır: {len(df):,}")
print(f"  Kolonlar: {df.shape[1]}")


ADIM 7: GEOLOCATION Bilgisini Ekle

Birleştirme sonrası:
  Satır: 110,189
  Kolonlar: 53


In [11]:
print("\n" + "=" * 80)
print(" YENİ FEATURE'LAR")
print("=" * 80)

# 1. Kâr marjı (ödeme - ürün fiyatı - kargo)
df['profit_margin'] = df['total_payment_value'] - df['price'] - df['freight_value']

# 2. Teslimat performansı (erken mi geç mi?)
df['delivery_performance'] = df['delivery_vs_estimated'].apply(
    lambda x: 'Early' if x < 0 else ('On Time' if x == 0 else 'Late')
)

# 3. Müşteri segmentasyonu (ödeme miktarına göre)
df['customer_segment'] = pd.cut(
    df['total_payment_value'],
    bins=[0, 50, 150, 500, float('inf')],
    labels=['Low', 'Medium', 'High', 'VIP']
)

# 4. Ürün boyut kategorisi
df['product_size_category'] = pd.cut(
    df['product_volume_cm3'],
    bins=[0, 1000, 10000, 100000, float('inf')],
    labels=['Small', 'Medium', 'Large', 'Extra Large']
)

# 5. Review kategorisi
df['review_category'] = pd.cut(
    df['avg_review_score'],
    bins=[0, 2, 3, 4, 5],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

# 6. Sipariş haftanın günü ve saati
df['order_day_of_week'] = pd.to_datetime(df['order_purchase_timestamp']).dt.day_name()
df['order_hour'] = pd.to_datetime(df['order_purchase_timestamp']).dt.hour

# 7. Taksit var mı?
df['has_installments'] = df['max_installments'] > 1

print("\n Yeni feature'lar oluşturuldu:")
print("""
  ✓ profit_margin - Kâr marjı
  ✓ delivery_performance - Teslimat performansı (Early/On Time/Late)
  ✓ customer_segment - Müşteri segmenti (Low/Medium/High/VIP)
  ✓ product_size_category - Ürün boyutu (Small/Medium/Large/XL)
  ✓ review_category - Yorum kategorisi (Poor/Fair/Good/Excellent)
  ✓ order_day_of_week - Siparişin günü
  ✓ order_hour - Siparişin saati
  ✓ has_installments - Taksitli mi?
""")


 YENİ FEATURE'LAR

 Yeni feature'lar oluşturuldu:

  ✓ profit_margin - Kâr marjı
  ✓ delivery_performance - Teslimat performansı (Early/On Time/Late)
  ✓ customer_segment - Müşteri segmenti (Low/Medium/High/VIP)
  ✓ product_size_category - Ürün boyutu (Small/Medium/Large/XL)
  ✓ review_category - Yorum kategorisi (Poor/Fair/Good/Excellent)
  ✓ order_day_of_week - Siparişin günü
  ✓ order_hour - Siparişin saati
  ✓ has_installments - Taksitli mi?



In [12]:
print("\n" + "=" * 80)
print(" FİNAL DATASET İNCELEMESİ")
print("=" * 80)

print(f"\n Final Dataset Boyutu: {df.shape}")
print(f"   - Satır: {len(df):,}")
print(f"   - Kolon: {df.shape[1]}")

print(f"\n Temel İstatistikler:")
print(f"   - Toplam Gelir: ${df['total_payment_value'].sum():,.2f}")
print(f"   - Ortalama Sipariş Değeri: ${df['total_payment_value'].mean():.2f}")
print(f"   - Medyan Sipariş Değeri: ${df['total_payment_value'].median():.2f}")
print(f"   - Ortalama Teslimat Süresi: {df['delivery_time_days'].mean():.1f} gün")
print(f"   - Ortalama Review Puanı: {df['avg_review_score'].mean():.2f}/5.00")

print(f"\n Top 5 Kategori (Gelir):")
top_categories = df.groupby('product_category_name')['total_payment_value'].sum().sort_values(ascending=False).head()
for cat, revenue in top_categories.items():
    print(f"   {cat}: ${revenue:,.2f}")

print(f"\n Top 5 Şehir (Müşteri):")
top_cities = df['customer_city'].value_counts().head()
for city, count in top_cities.items():
    print(f"   {city}: {count:,}")

print(f"\n Ödeme Tipi Dağılımı:")
payment_dist = df['payment_types'].value_counts().head()
for payment, count in payment_dist.items():
    print(f"   {payment}: {count:,}")

print(f"\n Teslimat Performansı:")
delivery_perf = df['delivery_performance'].value_counts()
for perf, count in delivery_perf.items():
    print(f"   {perf}: {count:,} ({count/len(df)*100:.1f}%)")

print(f"\n Müşteri Segmentasyonu:")
segment_dist = df['customer_segment'].value_counts()
for segment, count in segment_dist.items():
    print(f"   {segment}: {count:,} ({count/len(df)*100:.1f}%)")


 FİNAL DATASET İNCELEMESİ

 Final Dataset Boyutu: (110189, 61)
   - Satır: 110,189
   - Kolon: 61

 Temel İstatistikler:
   - Toplam Gelir: $19,774,781.52
   - Ortalama Sipariş Değeri: $179.47
   - Medyan Sipariş Değeri: $114.34
   - Ortalama Teslimat Süresi: 12.0 gün
   - Ortalama Review Puanı: 4.08/5.00

 Top 5 Kategori (Gelir):
   cama_mesa_banho: $1,692,714.28
   beleza_saude: $1,620,684.04
   informatica_acessorios: $1,549,252.47
   moveis_decoracao: $1,394,466.93
   relogios_presentes: $1,387,046.31

 Top 5 Şehir (Müşteri):
   SAO PAULO: 17,400
   RIO DE JANEIRO: 7,592
   BELO HORIZONTE: 3,087
   BRASILIA: 2,341
   CURITIBA: 1,727

 Ödeme Tipi Dağılımı:
   credit_card: 82,118
   boleto: 22,362
   debit_card: 1,651
   voucher: 1,613
   credit_card, voucher: 1,225

 Teslimat Performansı:
   Early: 101,475 (92.1%)
   Late: 7,264 (6.6%)
   On Time: 1,450 (1.3%)

 Müşteri Segmentasyonu:
   Medium: 53,166 (48.2%)
   High: 34,359 (31.2%)
   Low: 16,688 (15.1%)
   VIP: 5,973 (5.4%)


In [13]:
print("\n" + "=" * 80)
print(" EKSİK DEĞER KONTROLÜ")
print("=" * 80)

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

if len(missing_values) > 0:
    print("\n Eksik değerler var:")
    for col, count in missing_values.items():
        pct = (count / len(df)) * 100
        print(f"   {col}: {count:,} ({pct:.2f}%)")
else:
    print("\n Eksik değer yok!")



 EKSİK DEĞER KONTROLÜ

 Eksik değerler var:
   review_message: 64,226 (58.29%)
   product_name_lenght: 1,537 (1.39%)
   product_description_lenght: 1,537 (1.39%)
   product_photos_qty: 1,537 (1.39%)
   avg_comment_length: 827 (0.75%)
   avg_review_score: 827 (0.75%)
   review_category: 827 (0.75%)
   customer_geo_geolocation_lat: 288 (0.26%)
   customer_geo_geolocation_lng: 288 (0.26%)
   customer_geo_geolocation_city: 288 (0.26%)
   customer_geo_geolocation_state: 288 (0.26%)
   seller_geo_geolocation_lat: 249 (0.23%)
   seller_geo_geolocation_lng: 249 (0.23%)
   seller_geo_geolocation_city: 249 (0.23%)
   seller_geo_geolocation_state: 249 (0.23%)
   order_approved_at: 15 (0.01%)
   max_installments: 3 (0.00%)
   total_payment_value: 3 (0.00%)
   payment_types: 3 (0.00%)
   total_payments: 3 (0.00%)
   profit_margin: 3 (0.00%)
   customer_segment: 3 (0.00%)
   order_delivered_carrier_date: 1 (0.00%)


In [14]:
print("\n" + "=" * 80)
print(" FINAL DATASET KAYDET")
print("=" * 80)

# Tam veri setini kaydet
output_path = '../data/processed/final_merged_data.csv'
df.to_csv(output_path, index=False)

print(f"\n Final dataset kaydedildi!")
print(f" Lokasyon: {output_path}")
print(f" Boyut: {df.shape}")
print(f" Dosya boyutu: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


 FINAL DATASET KAYDET

 Final dataset kaydedildi!
 Lokasyon: ../data/processed/final_merged_data.csv
 Boyut: (110189, 61)
 Dosya boyutu: 170.12 MB


In [15]:
print("\n" + "=" * 80)
print(" KOLON DOKÜMANTASYONU")
print("=" * 80)

# Tüm kolonları ve tiplerini listele
column_info = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes.values,
    'Non-Null Count': df.count().values,
    'Null Count': df.isnull().sum().values
})

# Dokümantasyonu kaydet
doc_path = '../data/processed/column_documentation.csv'
column_info.to_csv(doc_path, index=False)

print(f"\n Kolon dokümantasyonu kaydedildi!")
print(f"Lokasyon: {doc_path}")

print("\n Tüm Kolonlar:")
print(column_info.to_string(index=False))


 KOLON DOKÜMANTASYONU

 Kolon dokümantasyonu kaydedildi!
Lokasyon: ../data/processed/column_documentation.csv

 Tüm Kolonlar:
                   Column Name      Data Type  Non-Null Count  Null Count
                      order_id         object          110189           0
                   customer_id         object          110189           0
                  order_status         object          110189           0
      order_purchase_timestamp datetime64[ns]          110189           0
             order_approved_at datetime64[ns]          110174          15
  order_delivered_carrier_date datetime64[ns]          110188           1
 order_delivered_customer_date datetime64[ns]          110189           0
 order_estimated_delivery_date datetime64[ns]          110189           0
                    order_year          int64          110189           0
                   order_month          int64          110189           0
                 order_quarter          int64          1101

In [16]:
print("\n" + "=" * 80)
print(" VERİ BİRLEŞTİRME TAMAMLANDI!")
print("=" * 80)

print("""
🎉 BAŞARIYLA TAMAMLANAN İŞLEMLER:

1. 8 farklı veri seti birleştirildi
2.  Tüm JOIN işlemleri tamamlandı
3.  8 yeni feature oluşturuldu
4.  Veri kalite kontrolleri yapıldı
5.  Final dataset kaydedildi
6.  Dokümantasyon oluşturuldu

 OLUŞTURULAN DOSYALAR:
  ✓ final_merged_data.csv - Master dataset
  ✓ column_documentation.csv - Kolon dokümantasyonu

 FINAL DATASET İSTATİSTİKLERİ:
""")
print(f"  • Toplam Satır: {len(df):,}")
print(f"  • Toplam Kolon: {df.shape[1]}")
print(f"  • Toplam Gelir: ${df['total_payment_value'].sum():,.2f}")
print(f"  • Zaman Aralığı: {df['order_purchase_timestamp'].min().date()} → {df['order_purchase_timestamp'].max().date()}")
print(f"  • Benzersiz Müşteri: {df['customer_unique_id'].nunique():,}")
print(f"  • Benzersiz Ürün: {df['product_id'].nunique():,}")
print(f"  • Benzersiz Kategori: {df['product_category_name'].nunique():,}")


 VERİ BİRLEŞTİRME TAMAMLANDI!

🎉 BAŞARIYLA TAMAMLANAN İŞLEMLER:

1. 8 farklı veri seti birleştirildi
2.  Tüm JOIN işlemleri tamamlandı
3.  8 yeni feature oluşturuldu
4.  Veri kalite kontrolleri yapıldı
5.  Final dataset kaydedildi
6.  Dokümantasyon oluşturuldu

 OLUŞTURULAN DOSYALAR:
  ✓ final_merged_data.csv - Master dataset
  ✓ column_documentation.csv - Kolon dokümantasyonu

 FINAL DATASET İSTATİSTİKLERİ:

  • Toplam Satır: 110,189
  • Toplam Kolon: 61
  • Toplam Gelir: $19,774,781.52
  • Zaman Aralığı: 2016-09-15 → 2018-08-29
  • Benzersiz Müşteri: 93,350
  • Benzersiz Ürün: 32,214
  • Benzersiz Kategori: 74
